In [ ]:
# Verzija 1.1
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from discord import Webhook, RequestsWebhookAdapter

from Harvest_data import Harvest_data
from Calculate_indicators import Calculate_indicators
from Trade_logic import Trade_logic
from Binance_API import Binance_API

## count_peaks bot

In [ ]:
#Splošne nastavitve
ticker1 = 'LUNA'
ticker2 = 'BUSD'
interval = '2h'

start_time = '2022-05-14 14:00:00' #Nastavi kdaj bo prva eksekucija bota

#Parametri izračuna
n = 22
peak = 12

#Trgovanje nastavitve
testnet = False #Izvajanje v testnem okolju. testnet=False - dejansko izvajanje ukazov
percentage = 1.0 #Delež vrednosti računa s katero trguje

#Log - nastavi pot in ime datoteke za shranjevanje
log_path = 'bot_log/20220512_count_peaks_2.csv'

#Discord webhook iniciacija
discord_webhook_url = '' #Vnesi svoj webhook naslov za discord server

webhook = Webhook.from_url(discord_webhook_url, adapter=RequestsWebhookAdapter())

In [ ]:
run_at = int(datetime.timestamp(datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')) * 1000) #Kdaj požene preračunana na unix
webhook.send('Bot ON!' + '\n' + 
             'Testnet: ' + str(testnet) + '\n' +
             'Prvi run_at: ' + start_time)

while True:
    state = ''
    now = int(datetime.timestamp(datetime.now())*1000)
    
    if now >= run_at:
        
        # Pridobitev in obdelava podatkov
        Korak1 = Harvest_data(ticker1, ticker2, start_date='2022-01-01', interval=interval)
        Korak1.get_data()
        Korak2 = Calculate_indicators(Korak1.data)
        Korak2.backtest_num_peaks(n = n)
        Korak2.EMA(prices='Close', name='Smooth_close', add_column = True, days= 4)
        Korak2.backtest_num_peaks(n = n, price='Smooth_close')
        Korak2.backtest_num_peaks_diff()

        Korak3 = Trade_logic(Korak2.data)
        Korak3.buy_and_hodl()
        Korak3.count_peaks_2(peak = peak)
        state = list(Korak3.data['Trades-count_peaks'])[-1]

        # Izvedba naročila
        Korak4 = Binance_API(ticker1, ticker2, testnet=testnet, log_path=log_path, discord_webhook_url=discord_webhook_url)
        Korak4.get_exchange_info()
    
        if state == 'BUY':
            order = Korak4.post_order(state, orderType='limit', timeInForce='GTC', percent=percentage)
            Korak4.log(order)
            try:    
                Korak4.send_discord_notification(order) #Pošlje sporočilo o transakciji na discord
            except:
                webhook.send('Ooops! Nekaj je narobe z post_order.\n' + order)
                break
            pass
        elif state == 'SELL':
            order = Korak4.post_order(state, orderType='limit', timeInForce='GTC', percent=percentage)
            Korak4.log(order)
            try:    
                Korak4.send_discord_notification(order) #Pošlje sporočilo o transakciji na discord
            except:
                webhook.send('Ooops! Nekaj je narobe z post_order.\n' + order)
                break
            pass
        elif state == 'None':
            pass
        else:
            print('Nekaj je narobe!')
            webhook.send('Nekaj je narobe! Bot ustavljen.')
            break
            
        run_at = now + (120 * 60 * 1000) #Preračuna kdaj naslednjič zaganati. (ponovno poženi čez 120 minut od zdaj)
        webhook.send('Tik!' + '\n' + 'Naslednji run_at: ' + datetime.strftime(datetime.fromtimestamp((run_at / 1000)), "%m/%d/%Y %H:%M:%S"))
        
        
    elif run_at < now: #Ustavi če se nekaj zacikla z uro
        print('Nekaj je narobe!')
        webhook.send('Nekaj je narobe! Bot ustavljen. Čas za run_at je manjši od now.')
        break
